In [1]:
from skimage.util import view_as_blocks
import numpy as np
import scipy.stats

def downsample_labels(a, factor):
    """
    Downsample the given ND array by binning the pixels into square bins
    (or cubic bins, etc.) sized according to the downsampling 'factor'.

    Each bin is reduced to one pixel in the output array, whose value
    is the most common element from the bin.
    """
    assert not (np.array(a.shape) % factor).any(), \
        "Downsampling factor must divide cleanly into array shape"
    v = view_as_blocks(a, (factor,)*a.ndim)
    v = v.reshape( v.shape[:a.ndim] + (factor**a.ndim,) )
    modes, _counts = scipy.stats.mode(v, axis=-1)
    return modes[...,0]

In [2]:
a = np.random.randint(0,4, (16,16))
print a

[[2 0 3 1 3 1 2 0 3 1 3 3 2 0 0 0]
 [2 2 3 3 1 3 1 0 2 3 1 3 3 3 2 3]
 [1 1 1 0 2 3 0 0 0 1 3 1 1 0 0 1]
 [0 3 0 3 1 1 1 0 2 0 0 2 2 1 1 1]
 [2 2 1 1 2 2 1 1 2 0 1 2 3 3 2 0]
 [0 0 3 1 0 2 1 3 2 2 2 2 1 3 0 3]
 [3 2 0 2 1 1 2 3 3 3 0 0 0 1 0 1]
 [1 2 0 0 2 0 3 0 1 0 3 1 0 3 1 2]
 [0 2 0 1 2 2 3 3 3 0 1 3 2 3 1 1]
 [3 1 3 1 1 1 1 3 3 2 3 1 1 3 1 1]
 [1 2 0 1 0 0 3 1 3 3 3 2 1 2 3 3]
 [0 1 2 2 0 3 2 0 0 3 0 0 1 1 2 1]
 [1 2 1 1 3 0 0 3 3 3 3 0 2 1 3 0]
 [0 0 2 2 0 0 0 0 1 2 2 0 1 2 2 3]
 [2 0 2 0 0 0 3 0 3 0 2 0 0 2 1 1]
 [2 2 3 1 1 3 3 0 0 0 3 2 1 3 2 1]]


In [3]:
print downsample_labels(a, 2)

[[2 3 1 0 3 3 3 0]
 [1 0 1 0 0 0 1 1]
 [0 1 2 1 2 2 3 0]
 [2 0 1 3 3 0 0 1]
 [0 1 1 3 3 1 3 1]
 [1 2 0 0 3 0 1 3]
 [0 1 0 0 3 0 1 3]
 [2 0 0 0 0 2 0 1]]
